<a href="https://colab.research.google.com/github/Akisazaki/af-sbp/blob/master/RSA_Implementation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
def FastModulo(b, p, d):
  i = 1
  while p > i: i = i << 1
  i = i >> 1
  r = 1
  while i > 0:
    r = (r * r) % d
    if p & i:
      r = (r * b) % d
    i = i >> 1
  return r

In [2]:
def GCD(m, n):
  if m < n:
    m, n = n, m
  if n == 0:
    return m
  else:
    return GCD(n, m % n)

In [3]:
# https://www.geeksforgeeks.org/euclidean-algorithms-basic-and-extended/
def GCDExtended(d, f):
  """
  Extended Euclidian Algorithm
  dd' = 1 mod f
  return gcd, csi, cbi
  """
  if d == 0:
    return f, 0, 1
  else:
    gcd, x, y = GCDExtended(f % d, d)
    return gcd, (y - (f // d) * x), x

In [4]:
def PrimeNumbers(index):
  """
  Too slow. throw it!
  """
  i = PrimeNumbers.primes[-1]
  while len(PrimeNumbers.primes) <= index:
    i = i + 2
    p = True
    for k in PrimeNumbers.primes:
      if i % k == 0:
        p = False
        break
    if p:
      PrimeNumbers.primes.append(i)
  return PrimeNumbers.primes[index]
PrimeNumbers.primes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]




#
# https://www.geeksforgeeks.org/how-to-generate-large-prime-numbers-for-rsa-algorithm/
#

# Large Prime Generation for RSA
import random
 
# Pre generated primes
first_primes_list = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29,
                     31, 37, 41, 43, 47, 53, 59, 61, 67,
                     71, 73, 79, 83, 89, 97, 101, 103,
                     107, 109, 113, 127, 131, 137, 139,
                     149, 151, 157, 163, 167, 173, 179,
                     181, 191, 193, 197, 199, 211, 223,
                     227, 229, 233, 239, 241, 251, 257,
                     263, 269, 271, 277, 281, 283, 293,
                     307, 311, 313, 317, 331, 337, 347, 349]
 
def nBitRandom(n):
    return random.randrange(2**(n-1)+1, 2**n - 1)
 
def getLowLevelPrime(n):
    '''Generate a prime candidate divisible
    by first primes'''
    while True:
        # Obtain a random number
        pc = nBitRandom(n)
 
         # Test divisibility by pre-generated
         # primes
        for divisor in first_primes_list:
            if pc % divisor == 0 and divisor**2 <= pc:
                break
        else: return pc
 
def isPrimeNumber(mrc):
    if mrc <= first_primes_list[-1]:
      return mrc in first_primes_list
    '''Run 20 iterations of Rabin Miller Primality test'''
    maxDivisionsByTwo = 0
    ec = mrc-1
    while ec % 2 == 0:
        ec >>= 1
        maxDivisionsByTwo += 1
    assert(2**maxDivisionsByTwo * ec == mrc-1)
 
    def trialComposite(round_tester):
        if pow(round_tester, ec, mrc) == 1:
            return False
        for i in range(maxDivisionsByTwo):
            if pow(round_tester, 2**i * ec, mrc) == mrc-1:
                return False
        return True
 
    # Set number of trials here
    numberOfRabinTrials = 20
    for i in range(numberOfRabinTrials):
        round_tester = random.randrange(2, mrc)
        if trialComposite(round_tester):
            return False
    return True

def nBitPrimeRandom(bits):
  while True:
    prime_candidate = getLowLevelPrime(bits)
    if isPrimeNumber(prime_candidate):
      return prime_candidate

In [5]:
# https://stackoverflow.com/questions/2654149/bit-length-of-a-positive-integer-in-python
def num_of_bits(n):
  return len('{:b}'.format(n))

In [6]:
def splitBit(v, n):
  kv = []
  while v > 0:
    t = v % n
    v = v // n
    kv.append(t)
  return kv

def mergeBit(bits, n):
  v = 0
  while len(bits) > 0:
    v = v * n
    v = v + bits.pop()
  return v

In [106]:
import math

def RSA(key_size = None, p = None, q = None):
  """
  p,q = prime numbers; p*q must be larger then 20.
  key_size = number of bits of p,q; must be larger then 2.
  """
  def nprime(n):
    if n < 10:
      m = 0
      while num_of_bits(first_primes_list[m]) < n: m = m + 1
      return PrimeNumbers(random.randrange(0, m + 1))
    return nBitPrimeRandom(n)
  
  def gcdExt(d, f): # using to calculate the d parameter
    gcd, csi, cbi = GCDExtended(d, f)
    if csi < 0:
      return f + csi # make positive
    else:
      return csi
  
  k = 0
  if None == key_size:
    if not isPrimeNumber(p) or not isPrimeNumber(q):
      raise Exception("p or q is not prime number.")
    # if p * q < 21:
    #   raise Exception("p or q is too small. p*q must be larger than 20.")

    k = max((num_of_bits(p) + num_of_bits(q)) // 2, 5)
  else:
    if key_size < 3:
      raise Exception("key_size is too small. key_size must be larger than 2 bit.")
    while True:
      p = nprime(key_size)
      q = nprime(key_size)
      if p != q:
        break
    k = max(key_size, 5)

  n = p * q
  pi = (p - 1) * (q - 1)
  e = nprime(k)
  d = -1
  try_count = 0
  while True:
    if math.log2(n) <= e and e != p and e != q and e != pi and GCD(e, pi) == 1: # check e // https://crypto.stackexchange.com/questions/87018/rsa-algorithm-must-e-be-less-than-varphin
      d = gcdExt(e, pi)
      if d > 1 and d != e:  # check d
        break
      elif 10 < try_count:
        raise Exception("Can't find acceptable d. Please try another p,q.")
      try_count = try_count + 1
    e = nprime(k)

  print("n:", n, " p:", p, " q:", q)
  print("e:", e, " d:", d)

  def enc(value):
    tokens = splitBit(value, n)
    result = []
    for token in tokens:
      result.append(FastModulo(token, e, n))
    return mergeBit(result, n)

  def dec(value):
    tokens = splitBit(value, n)
    result = []
    for token in tokens:
      result.append(FastModulo(token, d, n))
    return mergeBit(result, n)
  return enc, dec

In [109]:
enc, dec = RSA(key_size=3)

n: 10  p: 2  q: 5
e: 11  d: 3


In [110]:
cipher = enc(131)
print(cipher)
print(dec(cipher))

171
131


In [50]:
(13*1) % 2

1